# Data transformation

In [1]:
# Ignore warnings :
import warnings
warnings.filterwarnings('ignore')

# Data handling
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import itertools
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
diamonds=pd.read_csv('/home/carmencuadrado/Ironhack/ih_datamadpt0420_project_m2/diamonds_supercleaned.csv')

## Categorizing & simplifying numerical variables

In order to make diamonds dataframe more actionable we categorize **Carat** in gradual increasing steps creating the categorical type column *carat_scale*

In addition, we are going to calculate diamonds volume in order to substitude *x ,y & z* by *volume*

In [3]:
diamonds['carat_scale']=diamonds['carat'].apply(lambda x: "0-0.10" if x>0 and x<=0.10 else("0.10-0.25" if x>0.10 and x<=0.25 else ("0.25-0.50" if x>0.25 and x<=0.50 else ("0.50-0.75" if x>0.50 and x<=0.75 else ("0.75-1" if x>0.75 and x<=1 else ("1-1.25" if x >1 and x<=1.25 else ("1.25-1.50" if x>1.25 and x<=1.50 else ("1.50-1.75" if x>1.50 and x<=1.75 else ("1.75-2" if x>1.75 and x<=2 else ("2-2.5" if x>2 and x<=2.5 else ("2.5-3" if x>2.5 and x<3 else ("3-4" if x>3 and x<=4 else "+4"))))))))))))

## Enriching the dataframe

#### Attributing a cut to each diamond

According to its table, depth and length/width ratio each diamond have a different shape. In order to learn more about the diamonds given in this sample, we are going to infer its shape applying tha parameters found at diamonds.pro proportion guide. Those who does not fit into any class would be labelled as "fancy" diamonds

These are the different shapes present in the diamonds sample:

In [4]:
#Web scrapping 
url='https://www.diamonds.pro/guides/diamond-proportion/'
html = requests.get(url).content
soup = BeautifulSoup(html, 'lxml')
diamonds_proportions = soup.find_all('td')
x=diamonds_proportions[10:]

#Adapting what has been scrapped
split = 6
x = [x[i:i + split] for i in range(0, len(x), split)]

lista=[]
for i in range(0, len(x)):
    lista.append(x[i][0])
    lista.append(x[i][1])
    lista.append(x[i][2])
    lista.append(x[i][5])
    
lista2=[]
for i in lista:
    i=re.sub('\<.*?>','',str(i))
    x=re.sub('–\xa0','',str(i))
    x=re.sub('%','',str(i))
    lista2.append(x)
    
lista3=[]
s=4
lista3=[lista2[i:i + s] for i in range(0,len(lista2),s)]

lista4=[]
for i in range(0,len(lista3)):
    lista4.append(lista3[i][0])
    x=re.split('–|;|to',str(lista3[i][1]))
    for a in x:
        lista4.append(a)
    y=re.split('–|;|to',str(lista3[i][2]))
    for b in y:
        lista4.append(b)
    z=re.split('–|;|to',str(lista3[i][3]))
    for c in z:
        lista4.append(c)

lista5=[]
for i in lista4:
    i=re.sub('\xa0','',str(i))
    x=re.sub('&lt','',str(i))
    g=re.sub('\([a-z].*', '', str(x))
    lista5.append(g)
    
lista5=[i for i in lista5 if i]

lista6=[]
sp=7
a=[lista5[0:35][i:i +sp] for i in range(0,len(lista5[0:35]),sp)]
spl=6
b=[lista5[35:47][i:i +spl] for i in range(0,len(lista5[35:47]),spl)]
c=[lista5[47:54][i:i +sp] for i in range(0,len(lista5[47:54]),sp)]
d=lista5[54:60]
e=lista5[60:]

for i in a:
    lista6.append(i)
for i in b:
    lista6.append(i)
for i in c:
    lista6.append(i)
lista6.append(d)
lista6.append(e)

#Creating a parameter that would help us to identify diamonds' shape
diamonds['L/Wratio']=diamonds['x']/diamonds['y']

#stablishing the conditions of each shape
a=lista6[0]
b=lista6[1]
c=lista6[2]
d=lista6[3]
e=lista6[4]
f=lista6[5]
g=lista6[6]
h=lista6[7]
i=lista6[8]
j=lista6[9]


round1=diamonds['depth']>=float(a[1])
round2=diamonds['depth']<=float(a[2])
round3=diamonds['table']>=float(a[3])
round4=diamonds['table']<=float(a[4])
round5=diamonds['L/Wratio']>=float(a[5])
round6=diamonds['L/Wratio']<=float(a[6])
r=diamonds[round1&round2&round3&round4&round5&round6]

princess1=diamonds['depth']>=float(b[1])
princess2=diamonds['depth']<=float(b[2])
princess3=diamonds['table']>=float(b[3])
princess4=diamonds['table']<=float(b[4])
princess5=diamonds['L/Wratio']>=float(b[5])
princess6=diamonds['L/Wratio']<=float(b[6])
p=diamonds[princess1&princess2&princess3&princess4&princess5&princess6]

cushion1=diamonds['depth']>=float(c[1])
cushion2=diamonds['depth']<=float(c[2])
cushion3=diamonds['table']<=float(c[3])
cushion5=diamonds['L/Wratio']>=float(c[4])
cushion6=diamonds['L/Wratio']<=float(c[5])
cs=diamonds[cushion1&cushion2&cushion3&cushion5&cushion6]

cushion7=diamonds['L/Wratio']>=1.15
cushion8=diamonds['L/Wratio']<=float(c[6])
cu=diamonds[cushion1&cushion2&cushion3&cushion7&cushion8]

emerald=diamonds['depth']>=float(d[1])
emerald2=diamonds['depth']<=float(d[2])
emerald3=diamonds['table']>=float(d[3])
emerald4=diamonds['table']<=float(d[4])
emerald5=diamonds['L/Wratio']>=float(d[5])
emerald6=diamonds['L/Wratio']<=float(d[6])
em=diamonds[emerald&emerald2&emerald3&emerald4&emerald5&emerald6]

asscher=diamonds['depth']>=float(e[1])
asscher2=diamonds['depth']<=float(e[2])
asscher3=diamonds['table']>=float(e[3])
asscher4=diamonds['table']<=float(e[4])
asscher5=diamonds['L/Wratio']>=float(e[5])
asscher6=diamonds['L/Wratio']<=float(e[6])
ass=diamonds[asscher&asscher2&asscher3&asscher4&asscher5&asscher6]

oval=diamonds['depth']<=float(f[1])
oval3=diamonds['table']>=float(f[2])
oval4=diamonds['table']<=float(f[3])
oval5=diamonds['L/Wratio']>=float(f[4])
oval6=diamonds['L/Wratio']<=float(f[5])
o=diamonds[oval&oval3&oval4&oval5&oval6]

pear=diamonds['depth']<=float(g[1])
pear3=diamonds['table']>=float(g[2])
pear4=diamonds['table']<=float(g[3])
pear5=diamonds['L/Wratio']>=float(g[4])
pear6=diamonds['L/Wratio']<=float(g[5])
pe=diamonds[pear&pear3&pear4&pear5&pear6]

radiant=diamonds['depth']<=float(h[1])
radiant3=diamonds['table']>=float(h[2])
radiant4=diamonds['table']<=float(h[3])
radiant5=diamonds['L/Wratio']>=float(h[4])
radiant6=diamonds['L/Wratio']<=float(h[5])
ra=diamonds[radiant&radiant3&radiant4&radiant5&radiant6]

radiant1=diamonds['depth']<=float(h[1])
radiant13=diamonds['table']>=float(h[2])
radiant14=diamonds['table']<=float(h[3])
radiant15=diamonds['L/Wratio']>=1.20
radiant16=diamonds['L/Wratio']<=float(h[6])
rad=diamonds[radiant1&radiant13&radiant14&radiant15&radiant16]

heart=diamonds['depth']>=float(i[1])
heart2=diamonds['depth']<=float(i[2])
heart3=diamonds['table']>=float(i[3])
heart4=diamonds['table']<=float(i[4])
heart5=diamonds['L/Wratio']==float(i[5])
he=diamonds[heart&heart2&heart3&heart4&heart5]

marquise=diamonds['depth']>=float(j[1])
marquise2=diamonds['depth']<=float(j[2])
marquise3=diamonds['table']>=float(j[3])
marquise4=diamonds['table']<=float(j[4])
marquise5=diamonds['L/Wratio']>=float(j[5])
marquise6=diamonds['L/Wratio']<=float(j[6])
m=diamonds[marquise&marquise2&marquise3&marquise4&marquise5&marquise6]

diamonds['shape']=np.where(diamonds.index.isin(r.index), a[0], 'Fancy')
diamonds.loc[diamonds.index.isin((p.index)), 'shape'] = b[0]
diamonds.loc[diamonds.index.isin((cs.index)), 'shape'] = c[0]
diamonds.loc[diamonds.index.isin((cu.index)), 'shape'] = c[0]
diamonds.loc[diamonds.index.isin((em.index)), 'shape'] = d[0]
diamonds.loc[diamonds.index.isin((ass.index)), 'shape'] = e[0]
diamonds.loc[diamonds.index.isin((o.index)), 'shape'] = f[0]
diamonds.loc[diamonds.index.isin((pe.index)), 'shape'] = g[0]
diamonds.loc[diamonds.index.isin((ra.index)), 'shape'] = h[0]
diamonds.loc[diamonds.index.isin((rad.index)), 'shape'] = h[0]
diamonds.loc[diamonds.index.isin((he.index)), 'shape'] = i[0]
diamonds.loc[diamonds.index.isin((m.index)), 'shape'] = j[0]

diamonds['shape'].unique()

array(['Cushion Cut', 'Fancy', 'Radiant Cut', 'Round Brilliant Cut',
       'Pear Shape', 'Asscher Cut', 'Heart Shape', 'Emerald Cut'],
      dtype=object)

#### Matching each diamonds with a potential client

The following web pages from top luxury jewelry brands reveals the 4Cs standards of each of these top brands. Regarding this information we are going to classify our diamonds according each client standards, attributing a potential client to each of our diamonds. It is important to note that diamonds complying with the standards of the first potential clients, should be accepted by those of the bottom of the list who are less demanding and offer diamonds to a broader public

- Harry Winston: https://www.harrywinston.com/en/our-diamonds
- Cartier: https://www.cartier.com/en-us/collections/engagement/features/cartier-diamonds.html
- Van Cleef & Arpels: No information
- Buccellati:https://buccellati.com/en/4cs?
- Tiffany & Co.: https://www.tiffany.com/engagement/the-tiffany-guide-to-diamonds/
- Graff:https://www.graff.com/bridal/bridal-guides/the-4cs/
- Piaget: https://www.piaget.com/es-es/craftsmanship/diamonds
- Bvlgari: https://www.bulgari.com/en-int/engagement-and-wedding/bridal-jewelry/diamond-guide-gia-certificate.html
- Chopard:https://www.chopard.com/intl/chopard-diamonds

This is the outcome of the classification process:

In [5]:
hw_color=['D','E','F']
hw_clarity=['IF', 'VVS1', 'VVS2']
hw_cut=['Ideal','Premium']

c_color=['D','E','F','G','H']
c_clarity=['IF', 'VVS1', 'VVS2']
c_cut=['Ideal','Premium', 'Very good']

b_color=['D','E','F']
b_clarity=['IF', 'VVS1', 'VVS2']
b_cut=['Ideal','Premium', 'Very good']

t_color=['D','E','F','G','H','I']
t_clarity=['VS2', 'VS1', 'IF', 'VVS1', 'VVS2']
t_cut=['Ideal','Premium']

g_color=['D','E','F','G']
g_clarity=['VS2', 'VS1', 'IF', 'VVS1', 'VVS2']
g_cut=['Ideal','Premium', 'Very good']

p_color=['D','E','F']
p_clarity=['IF', 'VVS1', 'VVS2']
p_cut=['Ideal','Premium', 'Very good', 'Good']

bu_color=['D','E','F','G','H']
bu_clarity=['VS2', 'VS1', 'IF', 'VVS1', 'VVS2']
bu_cut=['Ideal','Premium', 'Very good']

ch_clarity=['VS2', 'VS1', 'IF', 'VVS1', 'VVS2']
ch_color=['D','E','F','G']
ch_cut=['Ideal','Premium']

condiciones = [diamonds.color.isin(hw_color) & diamonds.clarity.isin(hw_clarity) & diamonds.cut.isin(hw_cut),
               diamonds.color.isin(c_color) & diamonds.clarity.isin(c_clarity) & diamonds.cut.isin(c_cut),
               diamonds.color.isin(b_color) & diamonds.clarity.isin(b_clarity) & diamonds.cut.isin(b_cut),
               diamonds.color.isin(t_color) & diamonds.clarity.isin(t_clarity) & diamonds.cut.isin(t_cut),
               diamonds.color.isin(g_color) & diamonds.clarity.isin(g_clarity) & diamonds.cut.isin(g_cut),
               diamonds.color.isin(p_color) & diamonds.clarity.isin(p_clarity) & diamonds.cut.isin(p_cut),
               diamonds.color.isin(bu_color) & diamonds.clarity.isin(bu_clarity) & diamonds.cut.isin(bu_cut),
               diamonds.color.isin(ch_color) & diamonds.clarity.isin(ch_clarity) & diamonds.cut.isin(ch_cut)]
elecciones = ['Harry Winston', 'Cartier','Buccellati','Tiffany & Co', 'Graff','Piaget', 'Bulgari', 'Chopard']
diamonds["Potential Client"] = np.select(condiciones, elecciones, default="Other")

In [6]:
print( 'Potential Clients:', diamonds["Potential Client"].unique())

Potential Clients: ['Other' 'Tiffany & Co' 'Cartier' 'Harry Winston' 'Piaget']


## Ranking diamonds

Now we know and have enrinched our data frame we are ready to rank diamonds according to its quality

Starting sorting by numerical variables, we are going to prioritize by

- Carat_scale
- Volume

In [7]:
diamonds['volume']=diamonds['x']*diamonds['y']*diamonds['z']
diamonds.drop(columns=['x','y','z'], inplace=True)
diamonds['carat_scale'] = pd.Categorical(diamonds['carat_scale'], ["0-0.10","0.10-0.25","0.25-0.50","0.50-0.75","0.75-1","1-1.25","1.25-1.50","1.50-1.75","1.75-2","2-2.5","2.5-3","3-4","+4"])

In [8]:
diamonds_s=diamonds.sort_values(['carat_scale','volume'],ascending=(False,False))
diamonds_s.head()

,carat,cut,color,clarity,depth,table,price,carat_scale,L/Wratio,shape,Potential Client,volume
31068,4.50,Fair,J,I1,65.8,58.0,18531,+4,1.006890,Cushion Cut,Other,698.455296
27879,4.13,Fair,H,I1,64.8,61.0,17329,+4,1.015228,Radiant Cut,Other,633.355000
3467,4.01,Premium,I,I1,61.0,61.0,15223,+4,1.003960,Radiant Cut,Other,631.894380
23598,4.01,Premium,J,I1,62.5,62.0,15223,+4,1.008048,Radiant Cut,Other,621.496512
1143,3.00,Very Good,H,I1,63.1,55.0,6512,+4,1.014286,Cushion Cut,Other,484.639610


Continuing sorting by categorical variables, we are going to prioritize by

- Clarity
- Color
- Cut

In [9]:
diamonds['cut'] = pd.Categorical(diamonds['cut'], ["Fair", "Good","Very Good","Premium","Ideal"])
diamonds['clarity'] = pd.Categorical(diamonds['clarity'], ["I1", "I2","I3","SI1","SI2","VS1","VS2","VVS1","VVS2","IF"])
diamonds['color'] = pd.Categorical(diamonds['color'], ["I","J","H","G","F","E","D"])

In [10]:
diamonds_s=diamonds.sort_values(['clarity','color','cut'],ascending=(False,False,False))
diamonds_s.head()

,carat,cut,color,clarity,depth,table,price,carat_scale,L/Wratio,shape,Potential Client,volume
153,0.59,Ideal,D,IF,60.9,57.0,4208,0.50-0.75,0.994475,Fancy,Harry Winston,96.762600
2222,0.34,Ideal,D,IF,59.8,57.0,2346,0.25-0.50,1.004376,Round Brilliant Cut,Harry Winston,57.475062
3506,0.63,Ideal,D,IF,62.5,55.0,6549,0.50-0.75,0.994545,Fancy,Harry Winston,103.191550
3810,0.59,Ideal,D,IF,60.7,58.0,4161,0.50-0.75,0.992714,Fancy,Harry Winston,99.336060
8128,1.02,Ideal,D,IF,63.0,57.0,15575,1-1.25,1.006299,Cushion Cut,Harry Winston,162.711765


In both cases, our sorting methods yield similar dataframes. However, when we sort by price the outcome has nothing to do with a categorization according to diamond quality

In [11]:
diamonds_s.sort_values(['price'],ascending=False).head()

,carat,cut,color,clarity,depth,table,price,carat_scale,L/Wratio,shape,Potential Client,volume
7031,2.29,Premium,I,VS2,60.8,60.0,18823,2-2.5,1.003542,Fancy,Tiffany & Co,371.494200
3988,2.00,Very Good,G,SI1,63.5,56.0,18818,1.75-2,0.991217,Fancy,Other,317.333520
29092,1.51,Ideal,G,IF,61.7,55.0,18806,1.50-1.75,0.994602,Fancy,Cartier,249.029352
25517,2.29,Premium,I,SI1,61.8,59.0,18797,2-2.5,1.008284,Cushion Cut,Other,377.248560
618,2.00,Premium,I,VS1,60.8,59.0,18795,1.75-2,1.013716,Fancy,Tiffany & Co,320.144766


Having a look to the correlation of the new variables volume and L/Wratio to price, we could see how volume has a positively-close-to-one correlation relationship with price while L/Wratio has an irrelevant correlation. Consecuently, we are going to take volume into consideration for the final sorting, while L/Wratio will be dismissed

In [12]:
diamonds.corr()

,carat,depth,table,price,L/Wratio,volume
carat,1.000000,0.026856,0.182969,0.921862,0.148321,0.998978
depth,0.026856,1.000000,-0.293246,-0.014695,0.057821,0.009251
table,0.182969,-0.293246,1.000000,0.129665,0.141961,0.173597
price,0.921862,-0.014695,0.129665,1.000000,0.094292,0.923932
L/Wratio,0.148321,0.057821,0.141961,0.094292,1.000000,0.139437
volume,0.998978,0.009251,0.173597,0.923932,0.139437,1.000000


We will keep a price index in order to see how it differs from the diamonds quality ranking we yield

In [13]:
diamonds=diamonds.sort_values(['price'],ascending=False)
diamonds=diamonds.reset_index()

In [14]:
diamonds.rename(columns={'index':'Price index'})
diamonds.head()

,index,carat,cut,color,clarity,depth,table,price,carat_scale,L/Wratio,shape,Potential Client,volume
0,7031,2.29,Premium,I,VS2,60.8,60.0,18823,2-2.5,1.003542,Fancy,Tiffany & Co,371.494200
1,3988,2.00,Very Good,G,SI1,63.5,56.0,18818,1.75-2,0.991217,Fancy,Other,317.333520
2,29092,1.51,Ideal,G,IF,61.7,55.0,18806,1.50-1.75,0.994602,Fancy,Cartier,249.029352
3,25517,2.29,Premium,I,SI1,61.8,59.0,18797,2-2.5,1.008284,Cushion Cut,Other,377.248560
4,618,2.00,Premium,I,VS1,60.8,59.0,18795,1.75-2,1.013716,Fancy,Tiffany & Co,320.144766


Definitely sorting by numerical & categorical variables in this order:

- Clarity
- Color
- Carat_scale
- Volume
- Cut

In [15]:
diamonds.sort_values(['clarity','color','carat_scale','volume','cut'],ascending=(False,False,False,False,False)).reset_index()
diamonds.head()

,index,carat,cut,color,clarity,depth,table,price,carat_scale,L/Wratio,shape,Potential Client,volume
0,7031,2.29,Premium,I,VS2,60.8,60.0,18823,2-2.5,1.003542,Fancy,Tiffany & Co,371.494200
1,3988,2.00,Very Good,G,SI1,63.5,56.0,18818,1.75-2,0.991217,Fancy,Other,317.333520
2,29092,1.51,Ideal,G,IF,61.7,55.0,18806,1.50-1.75,0.994602,Fancy,Cartier,249.029352
3,25517,2.29,Premium,I,SI1,61.8,59.0,18797,2-2.5,1.008284,Cushion Cut,Other,377.248560
4,618,2.00,Premium,I,VS1,60.8,59.0,18795,1.75-2,1.013716,Fancy,Tiffany & Co,320.144766


In [16]:
diamonds.to_csv(r'/home/carmencuadrado/Ironhack/ih_datamadpt0420_project_m2/diamonds_result.csv')

In [17]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')